# MLlib

Spark biedt ook een framework aan voor MachineLearning modellen te trainen op gedistribueerde datasets.
Dit framework is MLlib of ook wel sparkML genoemd.
De code om te werken met deze package is sterk gelijkaardig aan sklearn.
De API en een uitgebreide documentatie met voorbeeldcode kan je [hier](https://spark.apache.org/docs/latest/ml-guide.html) vinden.

Deze package bied de volgende tools aan
* ML-technieken: classificatie, regressie, clustering, ...
* Features: Extracting en transforming van features, PCA, ...
* Pipelines: Maak, train, optimaliseer en evalueer pipelines
* Persistentie: Bewaar en laden van algoritmes/modellen
* Databeheer: Algebra tools, statistieken, null-waarden, ...

Let op dat er twee API's aangeboden worden, 1 gebaseerd op RDD's en 1 op DataFrames.
De API gebaseerd op RDD's is ouder en minder flexibel dan de API gebruik makend van DataFrames.
Momenteel werken ze allebei maar in de toekomst zou de RDD gebaseerde kunnen verdwijnen.

## Utilities

### Varianten voor numpy-arrays

Voor feature sets en volledige matrices van datasets aan te maken kan je gebruik maken van de Vector en Matrix klassen.
Deze beschikken over een Dense variant waar je elk element moet ingeven of een Sparse Variant waar cellen, elementen leeg kan laten.
Dit ziet er als volgt uit:

In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors, Matrices

spark = SparkSession.builder.appName("MLlib les").getOrCreate()

data = Vectors.sparse(4, [(1, 20.0), (3, 40.0)])
print(data)
data = Vectors.dense([10.0, 20.0, 30.0, 40.0])
print(data)

data = Matrices.dense(4, 5, range(20))
print(data)

(4,[1,3],[20.0,40.0])
[10.0,20.0,30.0,40.0]
DenseMatrix([[ 0.,  4.,  8., 12., 16.],
             [ 1.,  5.,  9., 13., 17.],
             [ 2.,  6., 10., 14., 18.],
             [ 3.,  7., 11., 15., 19.]])


Het is belangrijk om te weten dat dit locale datastructuren (wrapper rond numpy array) zijn en geen gedistribueerde objecten.

### Statistieken

Voor er kan gewerkt worden met statistieken moeten we (net zoals bij pandas) eerst een dataset hebben.
Hieronder maken we een random dataframe aan van 50 rijen en 4 kolommen.

In [11]:
from pyspark.mllib.random import RandomRDDs

data = RandomRDDs.uniformVectorRDD(spark.sparkContext, 50, 4).map(lambda a: a.tolist()).toDF()
#data.show()

data.describe().show()
data.summary().show()

+-------+--------------------+--------------------+-------------------+-------------------+
|summary|                  _1|                  _2|                 _3|                 _4|
+-------+--------------------+--------------------+-------------------+-------------------+
|  count|                  50|                  50|                 50|                 50|
|   mean|  0.5619601610654407| 0.49209999061944076|  0.521875319288421| 0.4590733343111752|
| stddev| 0.26506269062019394| 0.29904667441412913|0.29185161329470144|0.23896385789431646|
|    min|0.021379685980711383|0.003812825071723...|0.02943629334755138|0.03248279020730849|
|    max|  0.9851111772284917|  0.9912903124504471| 0.9910200179131231| 0.9271720371635891|
+-------+--------------------+--------------------+-------------------+-------------------+



+-------+--------------------+--------------------+-------------------+-------------------+
|summary|                  _1|                  _2|                 _3|                 _4|
+-------+--------------------+--------------------+-------------------+-------------------+
|  count|                  50|                  50|                 50|                 50|
|   mean|  0.5619601610654407| 0.49209999061944076|  0.521875319288421| 0.4590733343111752|
| stddev| 0.26506269062019394| 0.29904667441412913|0.29185161329470144|0.23896385789431646|
|    min|0.021379685980711383|0.003812825071723...|0.02943629334755138|0.03248279020730849|
|    25%|  0.3514710263444104| 0.19116167583430976| 0.2425225131579518|0.29996042545122203|
|    50%|  0.5745621026304127|  0.5347308147846938| 0.5479747750189802|0.47397828647392315|
|    75%|  0.7365141787009224|  0.7777935103406163| 0.7547012481420566| 0.6489217458144092|
|    max|  0.9851111772284917|  0.9912903124504471| 0.9910200179131231| 0.927172

**Correlation matrix**

Buiten de statistieken die berekend kunnen worden door de summary() functie kan ook de correlatiematrix belangrijk zijn.
Deze matrix maakt het mogelijk om het verband tussen de verscheidene features te bestuderen.
Deze matrix kan als volgt berekend worden voor een gedistribueerd dataframe.

In [16]:
# eerst alles omzetten naar vector
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=data.columns, outputCol="vector")
data_vector = assembler.transform(data)
data.show(5)
data_vector.show(5, truncate=False)

from pyspark.ml.stat import Correlation
df_corr = Correlation.corr(data_vector, "vector")
df_corr.show(truncate=False)

+------------------+------------------+-------------------+-------------------+
|                _1|                _2|                 _3|                 _4|
+------------------+------------------+-------------------+-------------------+
|0.7365141787009224|0.3168738538654351|0.43566569091158835| 0.7033486023579659|
|0.2437431531535279|0.5880173833841424| 0.7547012481420566| 0.2656352926280181|
|0.6771975979073663|0.5185951554693796| 0.6687511124589612| 0.4135632219689097|
|0.8589265556823864|0.6500408805605238| 0.8613772751634974|0.37799475244509406|
|0.6249239720137054|0.9912903124504471|0.02943629334755138|  0.716335492619434|
+------------------+------------------+-------------------+-------------------+
only showing top 5 rows

+------------------+------------------+-------------------+-------------------+------------------------------------------------------------------------------+
|_1                |_2                |_3                 |_4                 |vector           

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pearson(vector)                                                                                                                                                                                                                                                                                                                                               |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**Onafhankelijksheidtest**

Naast de correlatiematrix kan het ook belangrijk zijn om de onafhankelijkheid te testen tussen elke feature en een label.
Dit kan uitgevoerd worden door een zogenaamde ChiSquareTest.
Deze krijgt als input een dataframe, de naam van de kolom met de features (als vectors) en de naam van een kolom met de labels.
We kunnen deze test uitvoeren als volgt:

In [20]:
from pyspark.sql.functions import rand, when

# voeg een label toe op basis van een nieuw willekeurig getal
# we gaan kijken of het label afhankelijk is van de features in de vector kolom 
data_label = data_vector.withColumn("label", when(rand() > 0.5, 1).otherwise(0))
data_label.show(5)

from pyspark.ml.stat import ChiSquareTest
data_chi2test = ChiSquareTest.test(data_label, "vector", "label")
data_chi2test.show(truncate=False)

# goede features -> p waarde kleiner dan 0.05

+------------------+------------------+-------------------+-------------------+--------------------+-----+
|                _1|                _2|                 _3|                 _4|              vector|label|
+------------------+------------------+-------------------+-------------------+--------------------+-----+
|0.7365141787009224|0.3168738538654351|0.43566569091158835| 0.7033486023579659|[0.73651417870092...|    1|
|0.2437431531535279|0.5880173833841424| 0.7547012481420566| 0.2656352926280181|[0.24374315315352...|    0|
|0.6771975979073663|0.5185951554693796| 0.6687511124589612| 0.4135632219689097|[0.67719759790736...|    0|
|0.8589265556823864|0.6500408805605238| 0.8613772751634974|0.37799475244509406|[0.85892655568238...|    0|
|0.6249239720137054|0.9912903124504471|0.02943629334755138|  0.716335492619434|[0.62492397201370...|    0|
+------------------+------------------+-------------------+-------------------+--------------------+-----+
only showing top 5 rows



+------------------------------------------------------------------------------+----------------+-------------------------------------------------------------------------+
|pValues                                                                       |degreesOfFreedom|statistics                                                               |
+------------------------------------------------------------------------------+----------------+-------------------------------------------------------------------------+
|[0.4334366972557635,0.4334366972557635,0.4334366972557635,0.43343669725576317]|[49, 49, 49, 49]|[50.00000000000005,50.00000000000005,50.00000000000005,50.00000000000006]|
+------------------------------------------------------------------------------+----------------+-------------------------------------------------------------------------+



**Summarizer**

Andere statistieken per kolom kunnen berekend worden door gebruik te maken van de Summarizer klasse:

In [23]:
# mean count max
from pyspark.ml.stat import Summarizer

summarizer = Summarizer.metrics("mean", "count", "max")
data_vector.select(summarizer.summary(data_vector.vector)).show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|aggregate_metrics(vector, 1.0)                                                                                                                                   |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{[0.5619601610654409,0.4920999906194408,0.521875319288421,0.4590733343111752], 50, [0.9851111772284917,0.9912903124504471,0.9910200179131231,0.9271720371635891]}|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+



Het gebruik maken van de Summarizer maakt het dus mogelijk om rechtstreeks op de feature vectors te werken zonder ze eerst terug te moeten splitsen.

### Pipelines

Pipelines binnen Spark zijn een groep van high-level API's steunend op Dataframes om ML-pipelines aan te maken, optimaliseren en trainen.
De belangrijkste concepten binnen de Pipelines van Spark zijn:
* Dataframe: concept van de dataset
* Transformer: Zet een dataframe om in een ander dataframe
* Estimator: Zet een dataframe om in een model/transformer
* Pipeline: een ketting van transformers en estimators om een flow vast te leggen
* Parameter: API voor parameters van transformers en estimators aan te passen

Gebruik nu onderstaande mini-dataset waar we op basis van een tekstkolom met logistische regressie een bepaald label proberen te voorspellen.
Maak hiervoor een Pipeline uit die bestaat uit de volgende stappen:
* Tokenizer om de tekstkolom te splitsen in de overeenkomstige woorden
* HashingTf om de term frequency van de woorden te bepalen en het om te zetten naar een feature vector
* LogisticRegression Estimator om de voorspelling te doen.

Train daarna deze pipeline en maak de voorspellingen voor de traningsdata.
Hoe accuraat is dit model?

In [30]:
# Prepare training documents from a list of (id, text, label) tuples.
# Classificatie : zit spark in de tekst -> 1 is true, 0 is false
training = spark.createDataFrame([
    (0, "a a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF
from pyspark.ml.classification import LogisticRegression

# stap 1 -> splits tekst in list
tokenizer = Tokenizer(inputCol="text", outputCol="words")
# resultaat = [a, b, c, d, e, spark]
hasher = HashingTF(inputCol="words", outputCol="features")
# resultaat = (262144,[74920,89530,107107,148981,167694,173558],[1.0,1.0,1.0,1.0,1.0,1.0])
# dit is een sparse vector
# 262144 aantal elementen/mogelijke hashes
# woord met hash 74920 staat er 1 keer in, 89530 ook 1 keer in , ....
# dit is eigenlijk de bag of words
lr = LogisticRegression()

pipeline = Pipeline(stages=[tokenizer, hasher, lr])

#hasher.transform(tokenizer.transform(training)).show(truncate=False)
model = pipeline.fit(training)
predictions = model.transform(training)
predictions.show(truncate=False)

2022-03-24 10:53:11,188 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2022-03-24 10:53:11,189 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


+---+-----------------+-----+-------------------------+----------------------------------------------------------------------------+----------------------------------------+-----------------------------------------+----------+
|id |text             |label|words                    |features                                                                    |rawPrediction                           |probability                              |prediction|
+---+-----------------+-----+-------------------------+----------------------------------------------------------------------------+----------------------------------------+-----------------------------------------+----------+
|0  |a a b c d e spark|1.0  |[a, a, b, c, d, e, spark]|(262144,[74920,89530,107107,148981,167694,173558],[1.0,1.0,2.0,1.0,1.0,1.0])|[-19.141960614724113,19.141960614724113]|[4.86129672108177E-9,0.9999999951387033] |1.0       |
|1  |b d              |0.0  |[b, d]                   |(262144,[89530,148981],[1.0,1.0])    

### Evalueren van een model

In de pyspark.ml package zitten er ook functionaliteiten voor deze modellen te evalueren.
Meer informatie hierover vind je [hier](https://spark.apache.org/docs/2.2.0/mllib-evaluation-metrics.html).

In [31]:
# evalueren van het model
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

1.0

### Data sources

Door gebruik te maken van de sparkContext kunnen een reeks standaard databronnen ingelezen worden om datasets uit op te bouwen (Csv, Json, ...).
Daarnaast is het ook mogelijk om een folder met een reeks beelden te gebruiken als dataset om zo een model voor image classification te trainen.
Download nu [deze](https://www.kaggle.com/returnofsputnik/chihuahua-or-muffin) dataset en upload ze naar een folder op het hadoop filesysteem.

In [32]:
# download dataset
import opendatasets as od

od.download("https://www.kaggle.com/returnofsputnik/chihuahua-or-muffin")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jensbaetensodisee
Your Kaggle Key: ········


100%|████████████████████████████████████████| 183k/183k [00:00<00:00, 1.92MB/s]

In [33]:
# upload dataset
import pydoop.hdfs as hdfs

localFS = hdfs.hdfs(host='')
client = hdfs.hdfs(host='localhost', port=9000)

if not client.exists('/user/bigdata/06_Spark'):
    client.create_directory('/user/bigdata/06_Spark')
client.set_working_directory('/user/bigdata/06_Spark')
print(client.working_directory())
      
# upload input.txt
localFS.copy("chihuahua-or-muffin", client, "chihuahua-or-muffin")

2022-03-24 11:03:37,762 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/user/bigdata/06_Spark


0

De geuploade images kunnen nu ingelezen worden als volgt:

In [39]:
# read images
df = spark.read.format("image").option("dropInvalid",True).load("06_Spark/chihuahua-or-muffin")
df.show(2)
df.printSchema()
df.select("image.origin", "image.height", "image.width").show(truncate=False)

+--------------------+
|               image|
+--------------------+
|{hdfs://localhost...|
|{hdfs://localhost...|
+--------------------+
only showing top 2 rows

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

+-------------------------------------------------------------------------------+------+-----+
|origin                                                                         |height|width|
+-------------------------------------------------------------------------------+------+-----+
|hdfs://localhost:9000/user/bigdata/06_Spark/chihuahua-or-muffin/muffin-4.jpeg  |170   |172  |
|hdfs://localhost:9000/user/bigdata/06_Spark/chihuahua-or-muffin/muffin-7.jpeg  |172   |171  |
|hdfs://localhost:9000/user/bigdata/06_Spark/chihuahua-or-m

Merk op dat het werken met images niet zo eenvoudig is.
Hiervoor wordt binnen pyspark typisch gebruik gemaakt van de [sparkdl](https://smurching.github.io/spark-deep-learning/site/api/python/sparkdl.html) package.
Hierbij staat de dl voor deep learning.
Aangezien dit ons momenteel te ver leidt ga ik dit niet verder toelichten.

Een andere aparte databron die eenvoudig ingelezen kan worden is het formaat "libsvm".
Een bestand van dit formaat wordt ingelezen als een dataframe met twee kolommen: een label en een kolom met de feature-vectors.
De code om dergelijk bestand in te laden is:

In [ ]:
df = spark.read.format("libsvm").load("{path to file here}")